In [2]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
json_file_path = 'massive_test.data'

json_data_list = []


with open(json_file_path, 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            json_data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

test_df = pd.DataFrame(json_data_list)

test_df = test_df.drop(['indoml_id','partition','worker_id','slot_method','judgments','annot_utt'], axis = 1)
test_df = test_df.dropna()
test_df

,id,locale,scenario,utt
0,0,af-ZA,alarm,maak my wakker om vyf v. m. die week
1,3,af-ZA,audio,stil
2,8,af-ZA,iot,pienk is al wat ons nodig het
3,14,af-ZA,iot,en die donkerte het geval
4,19,af-ZA,iot,janneman skakel die ligte af in die slaapkamer
...,...,...,...,...
151669,17151,zh-TW,email,我的朋友禮安寄入厝邀請函回他說謝謝
151670,17156,zh-TW,email,檢查我的 email 有沒有任何來自小喬的新 email
151671,17157,zh-TW,email,我有任何小喬新寄的 email 嗎
151672,17170,zh-TW,email,唯楓寄電子郵件給我時通知我


In [4]:
le = LabelEncoder()
test_df['locale'] = le.fit_transform(test_df['locale'])
test_df['scenario'] = le.fit_transform(test_df['scenario'])
test_df

,id,locale,scenario,utt
0,0,0,0,maak my wakker om vyf v. m. die week
1,3,0,1,stil
2,8,0,7,pienk is al wat ons nodig het
3,14,0,7,en die donkerte het geval
4,19,0,7,janneman skakel die ligte af in die slaapkamer
...,...,...,...,...
151669,17151,50,5,我的朋友禮安寄入厝邀請函回他說謝謝
151670,17156,50,5,檢查我的 email 有沒有任何來自小喬的新 email
151671,17157,50,5,我有任何小喬新寄的 email 嗎
151672,17170,50,5,唯楓寄電子郵件給我時通知我


In [5]:
json_file_path = 'massive_train.data'

json_data_list = []


with open(json_file_path, 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            json_data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

train_df = pd.DataFrame(json_data_list)

train_df = train_df.drop(['partition','worker_id','slot_method','judgments','annot_utt'], axis = 1)
train_df = train_df.dropna()

train_df

,indoml_id,id,locale,scenario,utt
0,af-ZA|1,1,af-ZA,alarm,maak my wakker nege-uur v. m. op vrydag
1,af-ZA|2,2,af-ZA,alarm,stel 'n alarm vir twee ure van nou af
2,af-ZA|4,4,af-ZA,audio,janneman stilte
3,af-ZA|5,5,af-ZA,audio,stop
4,af-ZA|6,6,af-ZA,audio,janneman onderbreek dit vir tien sekondes
...,...,...,...,...,...
587209,zh-TW|17175,17175,zh-TW,email,用 line 傳 hi 給薇琪
587210,zh-TW|17176,17176,zh-TW,email,我有電子郵件嗎
587211,zh-TW|17177,17177,zh-TW,email,哪些是新電子郵件
587212,zh-TW|17178,17178,zh-TW,email,我有來自約翰的新電子郵件嗎


In [6]:
json_file_path = 'massive_train.solution'

json_data_list = []


with open(json_file_path, 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            json_data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

train_target = pd.DataFrame(json_data_list)
train_target

,indoml_id,intent
0,af-ZA|1,alarm_set
1,af-ZA|2,alarm_set
2,af-ZA|4,audio_volume_mute
3,af-ZA|5,audio_volume_mute
4,af-ZA|6,audio_volume_mute
...,...,...
587209,zh-TW|17175,email_querycontact
587210,zh-TW|17176,email_query
587211,zh-TW|17177,email_query
587212,zh-TW|17178,email_query


In [7]:
merged_train = train_df.merge(train_target, on = 'indoml_id')

le = LabelEncoder()
merged_train['locale'] = le.fit_transform(merged_train['locale'])
merged_train['intent'] = le.fit_transform(merged_train['intent'])
merged_train['scenario'] = le.fit_transform(merged_train['scenario'])

merged_train = merged_train.drop(['indoml_id','id'], axis = 1)

X_train = merged_train.drop('intent', axis = 1)
Y_train = pd.DataFrame(merged_train['intent'])
Y_train

,intent
0,2
1,2
2,4
3,4
4,4
...,...
587209,16
587210,15
587211,15
587212,15


In [8]:
json_file_path = 'massive_valid.data'

json_data_list = []


with open(json_file_path, 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            json_data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

valid_df = pd.DataFrame(json_data_list)

valid_df = valid_df.drop(['partition','worker_id','slot_method','judgments','annot_utt'], axis = 1)
valid_df = valid_df.dropna()

valid_df

,indoml_id,id,locale,scenario,utt
0,af-ZA|11,11,af-ZA,iot,skakel die ligte af asseblief
1,af-ZA|17,17,af-ZA,iot,verdof die ligte in die gang
2,af-ZA|24,24,af-ZA,iot,maak die vertrek donkerder
3,af-ZA|26,26,af-ZA,iot,maak die woonstel skoon
4,af-ZA|30,30,af-ZA,iot,skoonmaak is goed stof is so sleg doen nou jou...
...,...,...,...,...,...
103678,zh-TW|17121,17121,zh-TW,email,回報任何新的電子郵件
103679,zh-TW|17137,17137,zh-TW,email,我有幾則未讀訊息
103680,zh-TW|17140,17140,zh-TW,email,請回覆我剛已讀的 email
103681,zh-TW|17153,17153,zh-TW,email,瑋柏今天有寫信給我嗎


In [9]:
json_file_path = 'massive_valid.solution'

json_data_list = []


with open(json_file_path, 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            json_data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

valid_target = pd.DataFrame(json_data_list)
valid_target

,indoml_id,intent
0,af-ZA|11,iot_hue_lightoff
1,af-ZA|17,iot_hue_lightdim
2,af-ZA|24,iot_hue_lightdim
3,af-ZA|26,iot_cleaning
4,af-ZA|30,iot_cleaning
...,...,...
103678,zh-TW|17121,email_query
103679,zh-TW|17137,email_query
103680,zh-TW|17140,email_sendemail
103681,zh-TW|17153,email_query


In [10]:
merged_valid = valid_df.merge(valid_target, on = 'indoml_id')

le = LabelEncoder()
merged_valid['locale'] = le.fit_transform(merged_valid['locale'])
merged_valid['intent'] = le.fit_transform(merged_valid['intent'])
merged_valid['scenario'] = le.fit_transform(merged_valid['scenario'])

merged_valid = merged_valid.drop(['indoml_id','id'], axis = 1)

X_valid = merged_valid.drop(['intent'], axis = 1)
Y_valid = pd.DataFrame(merged_valid['intent'])
Y_valid

,intent
0,24
1,23
2,23
3,20
4,20
...,...
103678,14
103679,14
103680,16
103681,14


In [12]:
max_seq_length = 50
max_vocab_size = 10000  

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['utt'])
X_train_sequences = tokenizer.texts_to_sequences(X_train['utt'])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq_length, padding='post', truncating='post')

X_valid_sequences = tokenizer.texts_to_sequences(X_valid['utt'])
X_valid_padded = pad_sequences(X_valid_sequences, maxlen=max_seq_length, padding='post', truncating='post')

X_test_sequences = tokenizer.texts_to_sequences(test_df['utt'])
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_seq_length, padding='post', truncating='post')

Y_train = to_categorical(Y_train, num_classes=60)
Y_valid = to_categorical(Y_valid, num_classes=60)

model = Sequential([
    tf.keras.layers.Embedding(input_dim = max_vocab_size, output_dim = 128,input_length = max_seq_length),
    tf.keras.layers.LSTM(64, return_sequences = True),
    tf.keras.layers.GlobalAveragePooling1D(),
    Dense(60, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

X_train_padded = np.array(X_train_padded, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)

X_valid_padded = np.array(X_valid_padded, dtype=np.float32)
Y_valid = np.array(Y_valid, dtype=np.float32)

# print(X_train_padded)
model.fit(X_train_padded, Y_train, epochs=10, batch_size=128)

valid_loss, valid_accuracy = model.evaluate(X_valid,Y_valid)

# predicted_Y_train = model.predict(X_train)

MemoryError: Unable to allocate 7.88 GiB for an array with shape (587214, 60, 60) and data type float32

In [4]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/37/dc/399e63f5d1d96bb643404ee830657f4dfcf8503f5ba8fa3c6d465d0c57fe/urllib3-2.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ------------------- -------------------- 30.7/62.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 62.6/62.6 kB 844.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/158.3 kB ? eta -:--:--
   -----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth-oauthlib 1.0.0 requires requests-oauthlib>=0.7.0, which is not installed.
tabpy 2.8.0 requires twisted, which is not installed.
tensorflow-intel 2.13.0 requires wrapt>=1.11.0, which is not installed.
google-auth 2.23.0 requires urllib3<2.0, but you have urllib3 2.0.5 which is incompatible.


In [5]:
import pickle
import requests

In [13]:
with open('data.pkl','rb') as file:
    object = pickle.load(file)
    print(object)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config=None,
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_model_id=None,
hub_private_repo=False,
hub_str

     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/172.3 MB 1.9 MB/s eta 0:01:30
     ---------------------------------------- 0.2/172.3 MB 3.4 MB/s eta 0:00:51
     ---------------------------------------- 0.4/172.3 MB 3.3 MB/s eta 0:00:53
     ---------------------------------------- 0.5/172.3 MB 3.4 MB/s eta 0:00:51
     ---------------------------------------- 0.7/172.3 MB 3.5 MB/s eta 0:00:49
     ---------------------------------------- 0.8/172.3 MB 3.6 MB/s eta 0:00:48
     ---------------------------------------- 1.0/172.3 MB 3.7 MB/s eta 0:00:47
     ---------------------------------------- 1.1/172.3 MB 3.7 MB/s eta 0:00:46
     ---------------------------------------- 1.2/172.3 MB 3.8 MB/s eta 0:00:46
     ---------------------------------------- 1.4/172.3 MB 3.7 MB/s eta 0:00:47
     ---------------------------------------- 1.4/172.3 MB 3.7 MB/s eta 0:00:47
     ---------------------------------------- 1

In [5]:
pip install pandas


  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/502.5 kB ? eta -:--:--
   --- ------------------------------------ 41.0/502.5 kB ? eta -:--:--
   --- ------------------------------------ 41.0/502.5 kB ? eta -:--:--
   ----------- ---------------------------- 143.4/502.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 143.4/502.5 kB 1.4 MB/s eta 0:00:01
   ---------------------- ----------------- 276.5/502.5 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 276.5/502.5 kB 1.3 MB/s eta 0:00:01
   -------------------------------- ------- 409.6/502.5 kB 1.3 MB/s eta 0:00:01
   -------------------------------- ------- 409.6/502.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  501.8/502.5 kB 1.2 MB/s eta 0:00:01
   ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tabpy 2.8.0 requires twisted, which is not installed.


In [5]:
import pandas as pd

df = pd.read_csv("result.csv")
df['indoml_id'] = df['locale'] + '|' + df['id'].astype(str)
df = df.drop(['locale','id'], axis = 1)
df = df[['indoml_id','intent']]
df

df.to_csv('output.csv', sep = ',', index=False)

In [8]:
# Input JSON string with line breaks and whitespace
input_json = '''
{
    "indoml_id": "mn-MN|0",
    "intent": "alarm_set"
}
'''

# Remove line breaks and whitespace
output_json = input_json.replace('\n', '').replace(' ', '')

# Print the result
print(output_json)

{"indoml_id":"mn-MN|0","intent":"alarm_set"}


In [15]:
import json

with open('csvjson.json', 'r') as json_file:
    json_data = json.load(json_file)

if isinstance(json_data, list):
    individual_objects = []

    for item in json_data:
        individual_objects.append(item)

    with open('modified_data.json', 'w') as modified_file:
        for obj in individual_objects:
            json.dump(obj, modified_file)
            modified_file.write('\n')
else:
    print("The JSON data is not in the expected format.")